In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urdu-news-data/description.csv
/kaggle/input/urdu-news-data/headline.csv
/kaggle/input/urdu-stop-word/stopword.txt


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import numpy as np
import pandas as pd
import re

In [3]:
# Define maximum sequence length and number of classes
MAX_SEQUENCE_LENGTH = 300
NUM_CLASSES = 7  # Adjust this to your actual number of categories

In [4]:
# Load the CSV file
data = pd.read_csv('/kaggle/input/urdu-news-data/description.csv',engine='python',encoding='utf-8')

data = data.dropna()

In [5]:
stop_word_list = []
with open("/kaggle/input/urdu-stop-word/stopword.txt") as fin:
    for i in fin:
        stop_word_list.append(i.strip())
fin.close()

pattern = r'[a-zA-Z0-9!?:;"(,)]'
data['Description'] = data['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_word_list]))
data['Description'] = data['Description'].apply(lambda x: re.sub(pattern,'',x))

In [6]:
descriptions = data['Description'].tolist()
categories = data['Category'].tolist()

In [7]:
# Tokenize text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(descriptions)
sequences = tokenizer.texts_to_sequences(descriptions)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [8]:
# Encode categories
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(categories)
num_classes = len(label_encoder.classes_)


In [9]:
# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    padded_sequences, integer_encoded_labels, test_size=0.2
)


In [10]:
# Define the RNN model
model = tf.keras.Sequential([
    Embedding(5000, 128, input_length=MAX_SEQUENCE_LENGTH),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.SimpleRNN(32),
    Dense(NUM_CLASSES, activation="softmax"),
])

2023-12-15 15:31:17.103851: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 15:31:17.103993: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 15:31:17.104114: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 15:31:17.104237: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 15:31:17.104338: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 15:31:17.104620: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 15:31:17.104737: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

In [11]:
# Compile the model
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

In [12]:
# Train the model
model.fit(
    train_data,
    train_labels,
    epochs=10,
    validation_data=(test_data, test_labels),
)

Epoch 1/10
4431/4431 [==============================] - 552s 124ms/step - loss: 1.5032 - accuracy: 0.3917 - val_loss: 1.5740 - val_accuracy: 0.3509
Epoch 2/10
4431/4431 [==============================] - 553s 125ms/step - loss: 1.4640 - accuracy: 0.4146 - val_loss: 1.4326 - val_accuracy: 0.4198
Epoch 3/10
4431/4431 [==============================] - 550s 124ms/step - loss: 1.3813 - accuracy: 0.4570 - val_loss: 1.3808 - val_accuracy: 0.4523
Epoch 4/10
4431/4431 [==============================] - 551s 124ms/step - loss: 1.3603 - accuracy: 0.4653 - val_loss: 1.3301 - val_accuracy: 0.4785
Epoch 5/10
4431/4431 [==============================] - 548s 124ms/step - loss: 1.3747 - accuracy: 0.4612 - val_loss: 1.3842 - val_accuracy: 0.4519
Epoch 6/10
4431/4431 [==============================] - 548s 124ms/step - loss: 1.3682 - accuracy: 0.4666 - val_loss: 1.4163 - val_accuracy: 0.4512
Epoch 7/10
4431/4431 [==============================] - 549s 124ms/step - loss: 1.3794 - accuracy: 0.4621 - val_

In [13]:
# Predict on test data
predictions = model.predict(test_data)

1108/1108 [==============================] - 34s 31ms/step


In [14]:
# Generate classification report
y_pred = np.argmax(predictions, axis=1)
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       163
           1       0.49      0.71      0.58      6272
           2       0.21      0.06      0.10      6352
           3       0.36      0.44      0.40      5165
           4       0.27      0.11      0.16      6457
           5       0.77      0.63      0.70      5024
           6       0.29      0.54      0.37      6016

    accuracy                           0.40     35449
   macro avg       0.34      0.36      0.33     35449
weighted avg       0.38      0.40      0.37     35449



/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
